In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
# Function to preprocess yearly household data
def preprocess(df):
    # loop through 300 customers
    for i in range(1, 301):
        customer_data = df[df['Customer'] == i]
        # Remove unnecessary columns
        customer_data = customer_data.drop(columns=['Generator Capacity', 'Postcode'])

        # Reshape data into long format
        try:
            customer_data = pd.melt(customer_data, id_vars=['Customer', 'date', 'Consumption Category', 'Row Quality'], var_name='Time')
        except:
            customer_data = pd.melt(customer_data, id_vars=['Customer', 'date', 'Consumption Category'], var_name='Time')
        #customer_data.to_csv('customer_data.csv', index=False)

        # Zero pad date to format before conversion
        customer_data['date'] = customer_data['date'].astype(str)
        zero_padded_date = []
        for date in customer_data['date']:
            if len(date) < 9:
                new_date = '0' + date
                zero_padded_date.append(new_date)
            else:
                zero_padded_date.append(date)

        customer_data['date'] = zero_padded_date

        customer_data['Time'] = customer_data['Time'].astype(str)
        
        customer_data['datetime'] = customer_data['date'] + ' ' + customer_data['Time']

        # Convert to datetime format
        try:
            customer_data['datetime'] = pd.to_datetime(customer_data['datetime'], format='%d-%b-%y %H:%M')
        except:
            customer_data['datetime'] = pd.to_datetime(customer_data['datetime'], format='%d/%m/%Y %H:%M')
        customer_data = customer_data.sort_values('datetime')
        #customer_data.to_csv('customer_data_new.csv', index=False)

        # Create file name for each individual customer
        year = customer_data['datetime'].iloc[0].year
        year_interval = str(year) + '-' + str(year + 1)
        file_name = 'NL_' + year_interval + '_' + str(i) + '.csv'

        # Separate consumption categories
        gc_data = customer_data[customer_data['Consumption Category'] == 'GC']
        cl_data = customer_data[customer_data['Consumption Category'] == 'CL']
        gg_data = customer_data[customer_data['Consumption Category'] == 'GG']

        gc_data = gc_data.rename(columns = {'value': 'GC'})
        cl_data = cl_data.rename(columns = {'value': 'CL'})
        gg_data = gg_data.rename(columns = {'value': 'GG'})

        gc_data = gc_data.drop(columns=['Consumption Category'])
        cl_data = cl_data.drop(columns=['Consumption Category'])
        gg_data = gg_data.drop(columns=['Consumption Category'])

        gc_data.to_csv('data_gc.csv', index=False)
        cl_data.to_csv('data_cl.csv', index=False)
        gg_data.to_csv('data_gg.csv', index=False)

        # Compute net load depending on presence of CL
        if cl_data.empty == False:
            nl_data = pd.merge(gc_data, gg_data, on=['Customer', 'date', 'Time', 'datetime'], how='outer')
            nl_data = pd.merge(nl_data, cl_data, on=['Customer', 'date', 'Time', 'datetime'], how='outer')

            # Sort the merged data by datetime
            nl_data = nl_data.sort_values('datetime')

            # Fill NaN values with 0
            nl_data = nl_data.fillna(0)

            # Calculate net load
            nl_data['NL'] = nl_data['GC'] + nl_data['CL'] - nl_data['GG']
            nl_data.to_csv('processed_data/' + year_interval + '/' + file_name, index=False)
        else:
            nl_data = pd.merge(gc_data, gg_data, on=['Customer', 'date', 'Time', 'datetime'])
            nl_data['NL'] = nl_data['GC'] - nl_data['GG']
            nl_data.to_csv('processed_data/' + year_interval + '/' + file_name, index=False)

In [ ]:
# Test for detecting whether missing data is present
missing_data = pd.read_csv('raw_data/missing_data_test.csv', skiprows=1)
missing_values = missing_data.isnull()
print(missing_values.value_counts())

In [ ]:
y1_data = pd.read_csv('raw_data/2010-2011 Solar home electricity data.csv', skiprows=1)
y1_data.head()

In [ ]:
y2_data = pd.read_csv('raw_data/2011-2012 Solar home electricity data v2.csv', skiprows=1)
y3_data = pd.read_csv('raw_data/2012-2013 Solar home electricity data v2.csv', skiprows=1)

In [ ]:
missing_values = y1_data.isnull()
print(missing_values.value_counts())

In [ ]:
missing_values = y2_data.isnull()
print(missing_values.value_counts())

In [ ]:
missing_values = y3_data.isnull()
print(missing_values.value_counts())

In [ ]:
preprocess(y1_data)

In [ ]:
preprocess(y2_data)

In [ ]:
preprocess(y3_data)